# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [39]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from config_gkey4 import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [40]:
weather_data = pd.read_csv("../output_data/weather_data.csv")
weather_data.head()


,Unnamed: 0,City ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,2274275,Robertsport,6.7533,-11.3686,78.55,73,95,9.37,LR,1629050049
1,1,3372707,Ribeira Grande,38.5167,-28.7000,77.40,69,40,4.61,PT,1629050050
2,2,2110227,Butaritari,3.0707,172.7902,82.36,74,11,9.01,KI,1629050051
3,3,2075265,Busselton,-33.6500,115.3333,50.09,82,0,10.25,AU,1629050052
4,4,4030556,Rikitea,-23.1203,-134.9692,74.61,75,0,12.95,PF,1629050052


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [41]:
gmaps.configure(api_key=gkey)

In [42]:

# Store latitude and longitude in locations
locations = weather_data[["Latitude", "Longitude"]]
max_humid = weather_data["Humidity"].max()
max_humid 

humid = weather_data["Humidity"]

In [43]:
#Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=max_humid,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display map
fig 



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [44]:
weather_data_city_index = weather_data.set_index(["City"]).dropna()
weather_data_city_index

,Unnamed: 0,City ID,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City,,,,,,,,,,
Robertsport,0,2274275,6.7533,-11.3686,78.55,73,95,9.37,LR,1629050049
Ribeira Grande,1,3372707,38.5167,-28.7000,77.40,69,40,4.61,PT,1629050050
Butaritari,2,2110227,3.0707,172.7902,82.36,74,11,9.01,KI,1629050051
Busselton,3,2075265,-33.6500,115.3333,50.09,82,0,10.25,AU,1629050052
Rikitea,4,4030556,-23.1203,-134.9692,74.61,75,0,12.95,PF,1629050052
...,...,...,...,...,...,...,...,...,...,...
Poltava,545,696643,49.5937,34.5407,72.66,34,77,5.01,UA,1629050476
Teknāf,546,1185095,20.8624,92.3058,79.83,90,100,8.95,BD,1629050476
Phon,547,1607617,15.8160,102.5998,74.79,93,100,5.75,TH,1629050477


In [45]:
weather_criteria_df = weather_data_city_index.loc[(weather_data_city_index["Max Temp"]>70) & (weather_data_city_index["Max Temp"]<80)&(weather_data_city_index["Cloudiness"]<10)&(weather_data_city_index["Wind Speed"]<10)] 
weather_criteria_df = weather_criteria_df.reset_index()
weather_criteria_df

,City,Unnamed: 0,City ID,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Jamestown,9,5122534,42.0970,-79.2353,74.30,58,1,3.00,US,1629050056
1,Kapaa,16,5848280,22.0752,-159.3190,75.47,59,8,1.01,US,1629050062
2,Beloslav,30,725213,43.1833,27.7000,74.98,64,0,4.61,BG,1629050073
3,Rome,32,5134295,43.2128,-75.4557,76.51,49,1,5.75,US,1629049942
4,Toronto,50,6167865,43.7001,-79.4163,77.49,45,1,3.00,CA,1629050085
5,Plattsburgh,117,5131692,44.6995,-73.4529,74.50,41,0,1.99,US,1629050002
6,Horodyshche,126,708366,49.2849,31.4451,70.90,54,3,3.53,UA,1629050145
7,Rosario,220,3838583,-32.9468,-60.6393,70.61,37,0,1.01,AR,1629050217
8,Fredericton,225,5957776,45.9454,-66.6656,74.37,51,1,5.01,CA,1629050221
9,Yangi Marg`ilon,234,1513491,40.4272,71.7189,75.92,29,0,9.22,UZ,1629050228


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [46]:
hotel_df = pd.DataFrame(weather_criteria_df[["Latitude","Longitude","City","Country"]])
hotel_df["Hotel Name"] = "" 
hotel_df

,Latitude,Longitude,City,Country,Hotel Name
0,42.0970,-79.2353,Jamestown,US,
1,22.0752,-159.3190,Kapaa,US,
2,43.1833,27.7000,Beloslav,BG,
3,43.2128,-75.4557,Rome,US,
4,43.7001,-79.4163,Toronto,CA,
5,44.6995,-73.4529,Plattsburgh,US,
6,49.2849,31.4451,Horodyshche,UA,
7,-32.9468,-60.6393,Rosario,AR,
8,45.9454,-66.6656,Fredericton,CA,
9,40.4272,71.7189,Yangi Marg`ilon,UZ,


In [47]:
# Hotel variable
hotels = []

# Loop through hotels df to get nearest hotel
for index, row in hotel_df.iterrows():

    lat = row["Latitude"]
    lng = row["Longitude"]
    
    city_coords = f"{lat},{lng}"
    target_radius = 5000
    target_type = "lodging"
    
    my_params = {
        "location": city_coords,
        "radius": target_radius,
        "types": target_type,
        "key": gkey
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
  
    hotel_data = requests.get(base_url, params=my_params)
    #print(hotel_data.url)
    try:
        hotels.append(hotel_data["results"]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
hotel_df["Hotel Name"] = hotels
hotel_df

,Latitude,Longitude,City,Country,Hotel Name
0,42.0970,-79.2353,Jamestown,US,Nearest hotel not found
1,22.0752,-159.3190,Kapaa,US,Nearest hotel not found
2,43.1833,27.7000,Beloslav,BG,Nearest hotel not found
3,43.2128,-75.4557,Rome,US,Nearest hotel not found
4,43.7001,-79.4163,Toronto,CA,Nearest hotel not found
5,44.6995,-73.4529,Plattsburgh,US,Nearest hotel not found
6,49.2849,31.4451,Horodyshche,UA,Nearest hotel not found
7,-32.9468,-60.6393,Rosario,AR,Nearest hotel not found
8,45.9454,-66.6656,Fredericton,CA,Nearest hotel not found
9,40.4272,71.7189,Yangi Marg`ilon,UZ,Nearest hotel not found


In [48]:
#!pip install --upgrade gmaps

In [49]:
type(gkey)


str

In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [51]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))